# Build Dissect Network Packets

# What is a Computer Network ?

* A computer network is nothing but one or multiple connection between more than one computer
  * It can be wired or wireless


# Network packet or network frame

We generally comeacross two terminology packet and frame.

I would say for simplicity we can use Packet as a name.

But as per computer network 
- A Network frame is layer 2 packet
- A Network packet is layer 3 packet
- TCP header actually called as segments
  - (layer 5, 6, 7) (Application layers)

# Lets create a Mental Model of Layers Before Jump in to Scapy

# Let us Jump in to the actual agenda today

# Why Scapy ?

  * Its a powerfull framework to sniff, send, dissect or create network packets.
  * We can use this for scanning, sending probe requests or Simulating network attacks.
  * Scapy is more powerfull than other tools available for networking.




# Creating and modifying a packet

In [9]:
from scapy.all import *
target = "www.example.com"

In [10]:
packet = Ether()/IP(dst = target)/TCP()

In [12]:
packet.summary()

'Ether / IP / TCP 192.168.0.163:ftp_data > Net("www.example.com/32"):http S'

In [13]:
packet.show()

###[ Ethernet ]###
  dst       = None
  src       = 48:89:e7:34:ee:82
  type      = IPv4
###[ IP ]###
     version   = 4
     ihl       = None
     tos       = 0x0
     len       = None
     id        = 1
     flags     = 
     frag      = 0
     ttl       = 64
     proto     = tcp
     chksum    = None
     src       = 192.168.0.163
     dst       = Net("www.example.com/32")
     \options   \
###[ TCP ]###
        sport     = ftp_data
        dport     = http
        seq       = 0
        ack       = 0
        dataofs   = None
        reserved  = 0
        flags     = S
        window    = 8192
        chksum    = None
        urgptr    = 0
        options   = []



In [25]:
# Let us use for loop and build packets 
for packet in Ether()/IP()/TCP(dport = [22, 80, 443]):
    print(packet.summary())

Ether / IP / TCP 127.0.0.1:ftp_data > 127.0.0.1:ssh S
Ether / IP / TCP 127.0.0.1:ftp_data > 127.0.0.1:http S
Ether / IP / TCP 127.0.0.1:ftp_data > 127.0.0.1:https S


In [26]:
def print_frame_ether(frame):
    return frame[Ether].src
    

In [29]:
system_interfaces = get_if_list()


['lo', 'enp4s0', 'wlp5s0', 'docker0']

In [30]:
src_address = get_if_hwaddr("wlp5s0")

frame = Ether(src=src_address)/IP()/TCP(dport = [22, 80, 443])

In [31]:
print_frame_ether(frame)

'48:89:e7:34:ee:82'

# How to Sniff packet (capture packets) ?

In [45]:
packets = sniff(count = 5, filter = "arp")

In [46]:
packets.summary()

Ether / ARP who has 192.168.0.163 says 192.168.0.1 / Padding
Ether / ARP is at 48:89:e7:34:ee:82 says 192.168.0.163
Ether / ARP who has 192.168.0.163 says 192.168.0.1 / Padding
Ether / ARP is at 48:89:e7:34:ee:82 says 192.168.0.163
Ether / ARP who has 192.168.0.163 says 192.168.0.1 / Padding


In [47]:
packets[0]

<Ether  dst=48:89:e7:34:ee:82 src=50:0f:f5:31:16:f0 type=ARP |<ARP  hwtype=Ethernet (10Mb) ptype=IPv4 hwlen=6 plen=4 op=who-has hwsrc=50:0f:f5:31:16:f0 psrc=192.168.0.1 hwdst=00:00:00:00:00:00 pdst=192.168.0.163 |<Padding  load=b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00' |>>>

## So Far

- Brushed up some basics of Networking
- Built layers
- Print the packets
- Capture packets

## You havent shown us how to send the packet ? 

## Let us jump in little more depth Behind Everyday use of Internet 

# What will happen when we try to browse a website ?

![Http Request and Response](httpreq.png)

# Packet capture above request

![Pcap](http_pcap.png)

# The interesting part

When we try to send a http request,
- Browser send DNS request to fetch the ip of the domain in order to establish TCP Connection for HTTP request

DNS is the dependency for http request,
  - Why dont we talk directly to the Server ?
  - When the Browser use DNS ?




## What is clear now ?

- HTTP traffic is not just
  - Request and Response

* DNS --> TCP(SYN) --> TCP(SYN-ACK) --> TCP(ACK) --> HTTP(get) -->TCP(ACK) --> HTTP(Response)   

* --> TCP(ACK) --> TCP(FIN,ACK) --> TCP(FIN,ACK) --> TCP(ACK)


## Let us Jump in Send DNS request

In [72]:
domain = "icanhazip.com"
dport = 80
sport = RandShort()
iface = None # if you want to sepcify interface

dns_resp = sr1(IP(dst="8.8.8.8")/UDP(dport=53)/DNS(rd=1, qd=DNSQR(qname=domain)), verbose=0)
dst_ip = dns_resp[DNS].an.rdata
print(f"[+] {domain} resolved to {dst_ip}")

[+] icanhazip.com resolved to 104.16.185.241


## We got the Destination Let us create 3 TCP packets

In [73]:
ip = IP(dst=dst_ip)
syn = TCP(sport=sport, dport=dport, flags="S", seq=1000)
syn_ack = sr1(ip/syn, timeout=2, verbose=0)
print(syn_ack.seq)
if not syn_ack or syn_ack[TCP].flags != "SA":
    print("[-] SYN-ACK failed")
    exit()

ack = TCP(sport=sport, dport=dport, flags="A", seq=syn_ack.ack, ack=syn_ack.seq + 1)
send(ip/ack, verbose=0)
print("[+] TCP handshake complete")

1004061854
[+] TCP handshake complete


In [74]:
http_get = f"GET / HTTP/1.1\r\nHost: {domain}\r\nUser-Agent: Scapy\r\nConnection: close\r\n\r\n"
get_pkt = TCP(sport=sport, dport=dport, flags="PA", seq=ack.seq, ack=ack.ack)
send(ip/get_pkt/http_get, verbose=0)

print("[+] HTTP GET sent")

# I still need to build (I have to stop because of TCP RST sent by the kernel)

[+] HTTP GET sent


# I am going to Stop Here

# Questions ?

# Thank you

# Credits

* Chennaipy 
* Vijay and Shrayas 
* Scapy Documnetation and Tutorials
